In [1]:
# @title #安装下载必要的库
!git clone https://huggingface.co/spaces/supersolar/florence-sam-colab
from IPython.display import clear_output
clear_output()
!pip install -r /content/florence-sam-colab/requirements.txt
#!pip install gradio==5.1
from IPython.display import clear_output
clear_output()
%cd /content/florence-sam-colab
import os
from typing import Tuple, Optional
import cv2
import numpy as np
import supervision as sv
import torch
from PIL import Image
from tqdm import tqdm

from IPython.display import clear_output
clear_output()

%cd /content/
!git clone https://github.com/tangxiaobai/segment-anything-2-real-time
%cd /content/segment-anything-2-real-time
!pip install -e .
from IPython.display import clear_output
clear_output()
%cd /content/segment-anything-2-real-time/checkpoints
!chmod +x /content/segment-anything-2-real-time/checkpoints/download_sam2_ckpts.sh
!./content/segment-anything-2-real-time/checkpoints/download_sam2_ckpts.sh
!./download_sam2_ckpts.sh
clear_output()
%cd /content/florence-sam-colab
!pip install gradio==4.44.1
from IPython.display import clear_output
clear_output()

In [ ]:
# @title # 设置fps和加24帧到第一帧前
import cv2
import os
import numpy as np
from moviepy.editor import VideoFileClip, ImageClip, CompositeVideoClip  # Import CompositeVideoClip

def get_frame(video_path, frame_index):
    """获取指定帧的视频帧。"""
    cap = cv2.VideoCapture(video_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
    ret, frame = cap.read()
    cap.release()
    return frame

def change_fps_and_add_frame(video_path, output_path, new_fps=12):
    """更改视频的FPS并在开头添加第24帧。"""
    # 获取第24帧
    frame_24 = get_frame(video_path, 23)  # 帧索引从0开始，所以23对应第24帧
    clip = VideoFileClip(video_path)
    modified_clip = clip.set_fps(new_fps)

    # 从第24帧创建ImageClip
    frame_24_clip = ImageClip(frame_24).set_duration(1 / new_fps)  # 设置持续时间为1帧

    # 将第24帧剪辑和修改后的剪辑连接起来
    final_clip = CompositeVideoClip([frame_24_clip.set_start(0), modified_clip.set_start(frame_24_clip.duration)])

    # 写入输出文件
    final_clip.write_videofile(output_path, codec='libx264')
    print(f"视频创建成功: {output_path}")

def process_video(video_input, output_dir='/content/12fps/'):
    """处理视频并保存到指定目录。"""
    # 创建输出目录
    os.makedirs(output_dir, exist_ok=True)

    # 更改FPS并添加第24帧
    output_video = os.path.join(output_dir, os.path.basename(video_input))
    change_fps_and_add_frame(video_input, output_video)
    return output_video


# 示例调用
#video_input = '/content/小莫：看我变变变！莫雷高德VS费利克斯_338.mp4'# @param {type:"string"}
#output_video = process_video(video_input)
# 清除输出
from IPython.display import clear_output
clear_output()

In [3]:
# @title # 设置fps和加最中间的帧到第一帧前
import cv2
import os
import numpy as np
from moviepy.editor import VideoFileClip, ImageClip, CompositeVideoClip  # Import CompositeVideoClip

def get_frame(video_path, frame_index=None):
    """获取指定帧的视频帧或视频总帧数。"""
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if frame_index is None:
        # 如果未指定 frame_index，则返回总帧数
        cap.release()
        return total_frames
    else:
        # 否则返回指定帧
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
        ret, frame = cap.read()
        cap.release()
        return frame

def change_fps_and_add_frame(video_path, output_path, new_fps=12):
    """更改视频的FPS并在开头添加中间帧。"""
    # 获取视频总帧数
    total_frames = get_frame(video_path)

    # 计算中间帧索引
    middle_frame_index = total_frames // 2

    # 获取中间帧
    middle_frame = get_frame(video_path, middle_frame_index)
    clip = VideoFileClip(video_path)
    modified_clip = clip.set_fps(new_fps)

    # 从中间帧创建ImageClip
    middle_frame_clip = ImageClip(middle_frame).set_duration(1 / new_fps)  # 设置持续时间为1帧

    # 将中间帧剪辑和修改后的剪辑连接起来
    final_clip = CompositeVideoClip([middle_frame_clip.set_start(0), modified_clip.set_start(middle_frame_clip.duration)])

    # 写入输出文件
    final_clip.write_videofile(output_path, codec='libx264')
    print(f"视频创建成功: {output_path}")

def process_video(video_input, output_dir='/content/12fps/'):
    """处理视频并保存到指定目录。"""
    # 创建输出目录
    os.makedirs(output_dir, exist_ok=True)

    # 更改FPS并添加中间帧
    output_video = os.path.join(output_dir, os.path.basename(video_input))
    change_fps_and_add_frame(video_input, output_video)
    return output_video

#process_video('/content/小莫：看我变变变！莫雷高德VS费利克斯_307.mp4', output_dir='/content/12fps/')

In [4]:
# @title #设置提示词，获取坐标
import subprocess
import pickle

def set_prompt_and_get_coordinates(output_video, texts=['men', 'the table']):
    """
    设置提示词并获取坐标。

    参数:
    output_video (str): 输出视频的路径。
    texts (list): 提示词列表，默认为 ['men']。

    返回:
    all_ok_bboxes (subprocess.CompletedProcess): 命令执行的结果。
    """
        # Check if texts is a string and convert to a list if needed
    if isinstance(texts, str):
        texts = texts.split(',')  # Assuming prompts are separated by commas
        texts = [text.strip() for text in texts]
        print(texts)
    # 保存提示词到文件
    with open('/content/texts.pkl', 'wb') as file:
        pickle.dump(texts, file)

    # 构建命令
    command = ['python', '/content/segment-anything-2-real-time/f.py', output_video]

    # 执行命令并捕获输出
    all_ok_bboxes = subprocess.run(command, capture_output=True, text=True)

    return all_ok_bboxes

# 示例调用
#output_video = '/content/12fps/那年大赛的男双冠军含金量有多高？许昕张继科VS樊振东周雨！胸咚组合一路上打败了多少强敌？_498.mp4'
texts="men, the table,table tennis bat,ping pong ball"# @param {type:"string"}
#result = set_prompt_and_get_coordinates(output_video,texts)
#print(result.stdout)

In [5]:
# @title #sam2处理
import subprocess

def run_sam2(output_video):
    """
    运行 sam2 脚本处理视频。

    参数:
    - output_video (str): 输出视频的路径。

    返回:
    - subprocess.CompletedProcess: 命令执行的结果。
    """
    # 定义脚本路径
    script_path = '/content/segment-anything-2-real-time/script-sam2.py'

    # 将 all_ok_bboxes 序列化为 JSON 字符串
    all_ok_bboxes_json = 'all_ok_bboxes.stdout#json.dumps(all_ok_bboxes)'

    # 构建命令
    command = ['python3', script_path, '--bboxes', all_ok_bboxes_json, '--video', output_video]

    # 执行命令并捕获输出
    sam2_output = subprocess.run(command, capture_output=True, text=True)
    print(sam2_output)
    return sam2_output

# 示例调用
#output_video = 'path/to/output_video.mp4'
#result = run_sam2(output_video)
#print(result.stdout)

In [6]:
# @title #处理完了生成视频
import cv2
import os
import re
from moviepy.editor import VideoFileClip, ImageSequenceClip

def create_video_with_audio(image_folder, input_video_path):
    # 获取图像文件列表
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

    # 自然排序图像文件
    def natural_sort_key(s, _nsre=re.compile('([0-9]+)')):
        return [int(text) if text.isdigit() else text.lower() for text in re.split(_nsre, s)]

    image_files.sort(key=natural_sort_key)

    # 跳过第一张图片
    if image_files:
        image_files = image_files[1:]

    # 读取第一张图像以获取尺寸
    if image_files:
        first_image = cv2.imread(os.path.join(image_folder, image_files[0]))
        height, width, layers = first_image.shape
    else:
        raise ValueError("No valid images found in the folder after skipping the first one.")

    # 获取输入视频的帧率
    cap = cv2.VideoCapture(input_video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()

    # 创建图像序列视频
    image_paths = [os.path.join(image_folder, img) for img in image_files]
    clip = ImageSequenceClip(image_paths, fps=fps)

    # 从输入视频中提取音频
    audio_clip = VideoFileClip(input_video_path).audio

    # 将音频添加到视频中
    final_clip = clip.set_audio(audio_clip)

    # 生成与输入视频同名的输出文件
    output_video_path = os.path.join('/tmp/gradio/', os.path.basename(input_video_path))

    # 确保输出目录存在
    os.makedirs(os.path.dirname(output_video_path), exist_ok=True)

    # 导出最终视频
    final_clip.write_videofile(output_video_path, codec='libx264')

    print(f"Video created successfully: {output_video_path}")
    return output_video_path

# 示例调用
image_folder = '/content/output'
'''
input_video_path = output_video

create_video_with_audio(image_folder, input_video_path)
'''

'\ninput_video_path = output_video\n\ncreate_video_with_audio(image_folder, input_video_path)\n'

In [7]:
# @title #处理完了生成mask视频
import cv2
import os
import re
from moviepy.editor import VideoFileClip

def create_mask_video(image_folder, input_video_path):
    # 获取图像文件列表
    image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

    # 自然排序图像文件
    def natural_sort_key(s, _nsre=re.compile('([0-9]+)')):
        return [int(text) if text.isdigit() else text.lower() for text in re.split(_nsre, s)]

    image_files.sort(key=natural_sort_key)

    # 读取第一张图像以获取尺寸
    if image_files:
        first_image = cv2.imread(os.path.join(image_folder, image_files[0]))
        height, width, layers = first_image.shape
    else:
        raise ValueError("No valid images found in the folder after skipping the first one.")

    # 获取输入视频的帧率
    cap = cv2.VideoCapture(input_video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    cap.release()
    print(fps)

    # 创建视频编写器对象
    output_video_path = os.path.join('/tmp/gradio/', 'mask_' + os.path.basename(input_video_path))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # 将图像帧写入视频
    for image_file in image_files:
        image_path = os.path.join(image_folder, image_file)
        frame = cv2.imread(image_path)
        out.write(frame)

    out.release()
    return output_video_path



# 示例调用
image_folder = '/content/output_mask'
#input_video_path = output_video

#create_mask_video(image_folder, input_video_path)

In [ ]:
# @title #gradio api
import gradio as gr
import cv2
import os
import re
from moviepy.editor import VideoFileClip, vfx

def speed_up_video(input_video_path, speed_factor=2):
    """Speeds up a video by the given factor."""
    clip = VideoFileClip(input_video_path)
    sped_up_clip = clip.fx(vfx.speedx, speed_factor)
    output_video_path = os.path.join('/tmp/gradio/', "sped_up_" + os.path.basename(input_video_path))
    sped_up_clip.write_videofile(output_video_path, codec='libx264')
    print(f"Sped-up video created successfully: {output_video_path}")
    return output_video_path

def process_video(video_input):
    # 这里是你提供的处理视频的代码
    # 对 output_video 执行后续操作，例如 set_prompt_and_get_coordinates、run_sam2 等
    print('FFFF2222............................................')
    result = set_prompt_and_get_coordinates(video_input, texts)
    print(result.stdout)
    # 从文件中加载变量
    with open('/content/accflag.pkl', 'rb') as file:
        accflag = pickle.load(file)
    if accflag:
      print('no sam2,acc!!!')
      speed_up_video_path = speed_up_video(video_input, 2)
      return speed_up_video_path, speed_up_video_path
    else:
      print('sam2............................................')
      result = run_sam2(video_input)
      print(result.stdout)
      image_folder = '/content/output'
      ok_video = create_video_with_audio(image_folder, video_input)
      image_folder = '/content/output_mask'
      mask_video = create_mask_video(image_folder, video_input)
      ok_video_abs_path = os.path.abspath(ok_video)
      mask_video_abs_path = os.path.abspath(mask_video)
      print(ok_video_abs_path, mask_video_abs_path)
      return ok_video_abs_path, mask_video_abs_path




# 创建 Gradio 接口
iface = gr.Interface(
    fn=process_video,
    inputs=gr.Video(),
    outputs=[
        gr.File( label="OK Video"),
        gr.File( label="Mask Video")
    ],
    title="视频处理",
    description="上传视频，返回处理后的 OK 视频和 Mask 视频"
)

# 启动 Gradio 接口

# Launch the Gradio interface
iface.launch(share=True,debug=True,show_error=True )

In [8]:
import os
# ... (Your existing code) ...

# 获取视频文件列表
video_dir = '/content/'  # 视频文件目录
video_files = [f for f in os.listdir(video_dir) if f.endswith(('.mp4', '.avi', '.mov'))]  # 支持的视频格式

# 遍历处理每个视频
for video_file in video_files:
    print(f"Processing video: {video_file}")
    video_input = os.path.join(video_dir, video_file)
    output_video = process_video(video_input)

    # 对 output_video 执行后续操作，例如 set_prompt_and_get_coordinates、run_sam2 等
    result = set_prompt_and_get_coordinates(output_video, texts)
    print(result.stdout)
    result = run_sam2(output_video)
    print(result.stdout)
    create_video_with_audio('/content/output', output_video)
    create_mask_video('/content/output_mask', output_video)

    print(f"Finished processing video: {video_file}")


print("All videos processed.")

Processing video: 张本智和输阿鲁纳：乒乓球没有应该_212.mp4
Moviepy - Building video /content/12fps/张本智和输阿鲁纳：乒乓球没有应该_212.mp4.
MoviePy - Writing audio in 张本智和输阿鲁纳：乒乓球没有应该_212TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/12fps/张本智和输阿鲁纳：乒乓球没有应该_212.mp4



Moviepy - Done !
Moviepy - video ready /content/12fps/张本智和输阿鲁纳：乒乓球没有应该_212.mp4
视频创建成功: /content/12fps/张本智和输阿鲁纳：乒乓球没有应该_212.mp4
['men', 'the table', 'table tennis bat', 'ping pong ball']
['men', 'the table', 'table tennis bat', 'ping pong ball']
{'<OPEN_VOCABULARY_DETECTION>': {'bboxes': [[277.0450134277344, 120.35699462890625, 546.6470336914062, 393.5789794921875], [1188.3990478515625, 240.32699584960938, 1386.8790283203125, 566.9550170898438]], 'bboxes_labels': ['men', 'men'], 'polygons': [], 'polygons_labels': []}}
men add 1 1
men add 1 2
{'<OPEN_VOCABULARY_DETECTION>': {'bboxes': [[551.6090087890625, 218.65499877929688, 1183.43701171875, 396.67498779296875]], 'bboxes_labels': ['the table'], 'polygons': [], 'polygons_labels': []}}
men_count == 0 0
{'<OPEN_VOCABULARY_DETECTION>': {'bboxes': [[278.6990051269531, 212.46299743652344, 311.7790222167969, 247.29299926757812]], 'bboxes_labels': ['table tennis bat'], 'polygons': [], 'polygons_labels': []}}
men_count == 0 0
{'<OPEN_VOCABULARY_

MoviePy - Done.
Moviepy - Writing video /tmp/gradio/张本智和输阿鲁纳：乒乓球没有应该_212.mp4



Moviepy - Done !
Moviepy - video ready /tmp/gradio/张本智和输阿鲁纳：乒乓球没有应该_212.mp4
Video created successfully: /tmp/gradio/张本智和输阿鲁纳：乒乓球没有应该_212.mp4
12.0
Finished processing video: 张本智和输阿鲁纳：乒乓球没有应该_212.mp4
All videos processed.


In [10]:
from IPython.display import Video

video_path = '/tmp/gradio/张本智和输阿鲁纳：乒乓球没有应该_212.mp4'
display(Video(video_path, embed=True, width=640, height=480))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')